In [29]:
# stat Descriptive
# 1. Par groupe (π)
# 2. Taij D
# 3. inbreeding coef (F)
# 4. FROH/ROHcount (<100kb)
# 5. MAF
# 6. LD-decay
# 7. g. IBDsum/IBDcount => use ibdseq

# import scikit-allel
import os
import random
random.seed(42)
import time
import numpy as np
np.random.seed(42)
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
import pandas as pd
import zarr
import allel; print('scikit-allel', allel.__version__)
# check which version is installed
print(allel.__version__)


scikit-allel 1.3.5
1.3.5


In [ ]:
##### version 1

In [104]:
zarr_path = '/home/daron/bioInf/wilding/vcf_store/wilding_ag1000g.3R.merge.biallelic.zarr/'
callset = zarr.open_group(zarr_path, mode='r')

chrom = "3R"

# recode chrom pos in ndarray
#pos_path = '/'+chrom+'/variants/POS'
#pos = callset[pos_path]['POS']
#pos = allel.SortedIndex(callset[pos_path])


# create genotype array of whole dataset.
gt_path = '/'+chrom+'/calldata/GT'
gt = allel.GenotypeChunkedArray(callset[gt_path])

# get pop id
samples_path = '/'+chrom+'/samples'
samples_list = list(callset[samples_path])

#print(gt.shape)
#print(samples_list)

In [105]:
meta = pd.read_csv("/home/daron/bioInf/wilding/popstructure/popdifferenciation/fst/ag1000g_wilding.meta.txt", sep='\t')
meta

,ox_code,population
0,AA0040-C,GHcol
1,AA0041-C,GHcol
2,AA0042-C,GHcol
3,AA0043-C,GHcol
4,AA0044-C,GHcol
...,...,...
1214,LBV127,LBVwil
1215,LBV130,LBVwil
1216,LBV132,LBVwil
1217,LBV133,LBVwil


In [73]:
popList = ["BFcol", "GNcol", "GHcol", "CIcol", "LBVwil", "LPdom", "LPfor", "AOcol"]

In [106]:
dic = {}
k = 0

for i in range(0,len(popList)):
    for j in range(i+1,len(popList)):
        k=k+1
        dic[k] = {}
        
        subids_pop1 = meta.loc[meta['population'].isin([popList[i]])]
        samples_callset_index_pop1 = [samples_list.index(s) for s in subids_pop1['ox_code']]

        subids_pop2 = meta.loc[meta['population'].isin([popList[j]])]
        samples_callset_index_pop2 = [samples_list.index(s) for s in subids_pop2['ox_code']]

        subpops = [samples_callset_index_pop1, samples_callset_index_pop2]

        # calc FST average_weir_cockerham_fst
#        out = allel.average_weir_cockerham_fst(gt, subpops, 10000, max_allele=None)

        # calc FST average_hudson_fst
        gt_sub1 = gt.take(samples_callset_index_pop1, axis=1)
        ac_1 = gt_sub1.count_alleles(max_allele=3)

        gt_sub2 = gt.take(samples_callset_index_pop2, axis=1)
        ac_2 = gt_sub2.count_alleles(max_allele=3)

        out = allel.average_hudson_fst(ac_1, ac_2, blen=500)

        # calc zscore 
        mean = np.mean(out[3]) 
        zscore = (out[0]-mean)/np.std(out[3])

        dic[k]['pop1'] = popList[i]
        dic[k]['pop2'] = popList[j]
        dic[k]['fst'] = out[0]
        dic[k]['se'] = out[1]
        dic[k]['zscore'] = zscore
#        dic[k]['vb'] = out[2]
#        dic[k]['vj'] = out[3]


print(dic)

{1: {'pop1': 'BFcol', 'pop2': 'GNcol', 'fst': 0.04735895450506136, 'se': 0.0007312993567736571, 'zscore': 1.4141230104416747}, 2: {'pop1': 'BFcol', 'pop2': 'GHcol', 'fst': 0.014248666801566067, 'se': 0.0003834790479066024, 'zscore': 5.375236723151047}, 3: {'pop1': 'BFcol', 'pop2': 'CIcol', 'fst': 0.013096210307242182, 'se': 0.0003302922949223459, 'zscore': 3.5765661975920016}, 4: {'pop1': 'BFcol', 'pop2': 'LBVwil', 'fst': 0.07334330088658018, 'se': 0.000882229377697959, 'zscore': 5.952929061020662}, 5: {'pop1': 'BFcol', 'pop2': 'LPdom', 'fst': 0.05814216013086725, 'se': 0.0007934492566226165, 'zscore': 11.220836980413258}, 6: {'pop1': 'BFcol', 'pop2': 'LPfor', 'fst': 0.05753242842264906, 'se': 0.0007777101364339046, 'zscore': 9.596312696238904}, 7: {'pop1': 'BFcol', 'pop2': 'AOcol', 'fst': 0.12672398725332434, 'se': 0.00150809597411402, 'zscore': 8.46690697104096}, 8: {'pop1': 'GNcol', 'pop2': 'GHcol', 'fst': 0.044574477953687734, 'se': 0.000613958081435808, 'zscore': 0.164263319023507

In [107]:
output = pd.DataFrame.from_dict(dic)
np.savetxt("/home/daron/bioInf/wilding/popstructure/popdifferenciation/fst/ag1000g_wilding.fst.matrix.txt", output.T, newline="\n", delimiter="\t", fmt='%s')
output.T

,pop1,pop2,fst,se,zscore
1,BFcol,GNcol,0.047359,0.000731,1.414123
2,BFcol,GHcol,0.014249,0.000383,5.375237
3,BFcol,CIcol,0.013096,0.00033,3.576566
4,BFcol,LBVwil,0.073343,0.000882,5.952929
5,BFcol,LPdom,0.058142,0.000793,11.220837
6,BFcol,LPfor,0.057532,0.000778,9.596313
7,BFcol,AOcol,0.126724,0.001508,8.466907
8,GNcol,GHcol,0.044574,0.000614,0.164263
9,GNcol,CIcol,0.04015,0.000519,-0.829003
10,GNcol,LBVwil,0.109394,0.00118,-0.031558


In [ ]:
#### version 2

In [87]:
zarr_path = '/home/daron/bioInf/wilding/vcf_store/wilding_ag1000g.3.merge.biallelic.zarr/'
callset = zarr.open_group(zarr_path, mode='r')

In [92]:
popList = ["BFcol", "GNcol", "GHcol", "CIcol", "LBVwil", "LPdom", "LPfor", "AOcol"]

In [101]:
dic = {}
k = 0

for i in range(0,len(popList)):
    for j in range(i+1,len(popList)):
        key=popList[i]+"_"+popList[j]
        dic[key] = {}
        dic[key]['fstArray'] = []
        dic[key]['bkArray'] = []
        
        for c in ["3L", "3R"]:
            # create genotype array
            chrom = c
            gt_path = '/'+chrom+'/calldata/GT'
            gt = allel.GenotypeChunkedArray(callset[gt_path])    

            subids_pop1 = meta.loc[meta['population'].isin([popList[i]])]
            samples_callset_index_pop1 = [samples_list.index(s) for s in subids_pop1['ox_code']]

            subids_pop2 = meta.loc[meta['population'].isin([popList[j]])]
            samples_callset_index_pop2 = [samples_list.index(s) for s in subids_pop2['ox_code']]


            # calc FST average_weir_cockerham_fst
    #        subpops = [samples_callset_index_pop1, samples_callset_index_pop2]
    #        out = allel.average_weir_cockerham_fst(gt, subpops, 10000, max_allele=None)

            # calc FST average_hudson_fst
            gt_sub1 = gt.take(samples_callset_index_pop1, axis=1)
            ac_1 = gt_sub1.count_alleles(max_allele=3)

            gt_sub2 = gt.take(samples_callset_index_pop2, axis=1)
            ac_2 = gt_sub2.count_alleles(max_allele=3)

            out = allel.average_hudson_fst(ac_1, ac_2, blen=1000)

            # calc zscore 
            mean = np.mean(out[3]) 
            zscore = (out[0]-mean)/np.std(out[3])

            dic[key]['pop1'] = popList[i]
            dic[key]['pop2'] = popList[j]
            dic[key]['fstArray'].append(out[0])
            dic[key]['bkArray'] = np.concatenate((dic[key]['bkArray'], out[3]))


print(dic)

{'BFcol_GNcol': {'fstArray': [0.050907766579159325, 0.050907766579159325], 'bkArray': array([0.05080598, 0.05089552, 0.05085583, ..., 0.05087128, 0.05086415,
       0.05087034]), 'pop1': 'BFcol', 'pop2': 'GNcol'}, 'BFcol_GHcol': {'fstArray': [0.01625540361267175, 0.01625540361267175], 'bkArray': array([0.01617616, 0.01617434, 0.01618295, ..., 0.01620381, 0.01621846,
       0.01619714]), 'pop1': 'BFcol', 'pop2': 'GHcol'}, 'BFcol_CIcol': {'fstArray': [0.014929920081523633, 0.014929920081523633], 'bkArray': array([0.01487907, 0.01488835, 0.01490511, ..., 0.01490606, 0.01490394,
       0.01490143]), 'pop1': 'BFcol', 'pop2': 'CIcol'}, 'BFcol_LBVwil': {'fstArray': [0.07314327636902807, 0.07314327636902807], 'bkArray': array([0.07292914, 0.07302038, 0.07302413, ..., 0.07305135, 0.07307165,
       0.07303886]), 'pop1': 'BFcol', 'pop2': 'LBVwil'}, 'BFcol_LPdom': {'fstArray': [0.05835488507761967, 0.05835488507761967], 'bkArray': array([0.05806761, 0.05820215, 0.05814572, ..., 0.05818885, 0.0581

In [102]:
for key in dic.keys():
    dic[key]['fst'] = np.mean(dic[key]['fstArray'])
    dic[key]['zscore'] = (dic[key]['fst']-np.mean(dic[key]['bkArray']))/np.std(dic[key]['bkArray'])
    
print(dic)


{'BFcol_GNcol': {'fstArray': [0.050907766579159325, 0.050907766579159325], 'bkArray': array([0.05080598, 0.05089552, 0.05085583, ..., 0.05087128, 0.05086415,
       0.05087034]), 'pop1': 'BFcol', 'pop2': 'GNcol', 'fst': 0.050907766579159325, 'zscore': 0.608907237453275}, 'BFcol_GHcol': {'fstArray': [0.01625540361267175, 0.01625540361267175], 'bkArray': array([0.01617616, 0.01617434, 0.01618295, ..., 0.01620381, 0.01621846,
       0.01619714]), 'pop1': 'BFcol', 'pop2': 'GHcol', 'fst': 0.01625540361267175, 'zscore': 1.3383638197492311}, 'BFcol_CIcol': {'fstArray': [0.014929920081523633, 0.014929920081523633], 'bkArray': array([0.01487907, 0.01488835, 0.01490511, ..., 0.01490606, 0.01490394,
       0.01490143]), 'pop1': 'BFcol', 'pop2': 'CIcol', 'fst': 0.014929920081523633, 'zscore': 0.8148309736613134}, 'BFcol_LBVwil': {'fstArray': [0.07314327636902807, 0.07314327636902807], 'bkArray': array([0.07292914, 0.07302038, 0.07302413, ..., 0.07305135, 0.07307165,
       0.07303886]), 'pop1': 'B

In [103]:
output = pd.DataFrame.from_dict(dic)
np.savetxt("/home/daron/bioInf/wilding/popstructure/popdifferenciation/fst/ag1000g_wilding.fst.matrix.v2.txt", output.T[["pop1", "pop2", "fst", "zscore"]], newline="\n", delimiter="\t", fmt='%s')
output.T[["pop1", "pop2", "fst", "zscore"]]

,pop1,pop2,fst,zscore
BFcol_GNcol,BFcol,GNcol,0.050908,0.608907
BFcol_GHcol,BFcol,GHcol,0.016255,1.338364
BFcol_CIcol,BFcol,CIcol,0.01493,0.814831
BFcol_LBVwil,BFcol,LBVwil,0.073143,2.653093
BFcol_LPdom,BFcol,LPdom,0.058355,4.816723
BFcol_LPfor,BFcol,LPfor,0.057866,4.096995
BFcol_AOcol,BFcol,AOcol,0.123522,3.757788
GNcol_GHcol,GNcol,GHcol,0.045549,0.038752
GNcol_CIcol,GNcol,CIcol,0.041483,-0.407903
GNcol_LBVwil,GNcol,LBVwil,0.109003,-0.044179


In [99]:
output.T[["pop1", "pop2", "fst", "zscore"]]

,pop1,pop2,fst,zscore
BFcol_GNcol,BFcol,GNcol,0.050908,0.486172
BFcol_GHcol,BFcol,GHcol,0.016255,0.711508
BFcol_CIcol,BFcol,CIcol,0.01493,0.427509
BFcol_LBVwil,BFcol,LBVwil,0.073143,0.690072
BFcol_LPdom,BFcol,LPdom,0.058355,1.667866
BFcol_LPfor,BFcol,LPfor,0.057866,1.293983
BFcol_AOcol,BFcol,AOcol,0.123522,1.207365
GNcol_GHcol,GNcol,GHcol,0.045549,0.639287
GNcol_CIcol,GNcol,CIcol,0.041483,0.391092
GNcol_LBVwil,GNcol,LBVwil,0.109003,0.173411


In [13]:
## Select given populations
pop1 = pd.read_csv("/home/daron/bioInf/wilding/popstructure/popdifferenciation/fst/LPdom.ind", sep='\t', header=None)
pop2 = pd.read_csv("/home/daron/bioInf/wilding/popstructure/popdifferenciation/fst/LPfor.ind", sep='\t', header=None)
pop3 = pd.read_csv("/home/daron/bioInf/wilding/popstructure/popdifferenciation/fst/LBVwil.ind", sep='\t', header=None)

samples_callset_index1 = [samples_list.index(s) for s in pop1[0]]
samples_callset_index2 = [samples_list.index(s) for s in pop2[0]]
samples_callset_index3 = [samples_list.index(s) for s in pop3[0]]

subpops_LPdom_LPfor = [samples_callset_index1, samples_callset_index2]
subpops_LPdom_LBVwil = [samples_callset_index1, samples_callset_index3]
subpops_LPfor_LBVwil = [samples_callset_index2, samples_callset_index3]

print(subpops_LPdom_LPfor)


[[1203, 1204, 1205, 1206, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189], [1210, 1213, 1214, 1215, 1216, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199, 1200, 1201, 1202, 1207, 1208, 1209, 1211, 1212, 1217, 1218]]


In [33]:
# calc FST LPdom vs LPfor
allel.average_weir_cockerham_fst(gt, subpops_LPdom_LPfor, 50000, max_allele=None)

(0.00026240982897839213,
 0.00044568410835224787,
 array([ 0.00243546,  0.00392056, -0.00268097,  0.00120027,  0.00103319,
        -0.0002243 , -0.00064639,  0.000198  ,  0.00098337, -0.00182138,
        -0.00011965,  0.00247855, -0.00514772, -0.00209815, -0.00220233,
        -0.00096832, -0.00030609,  0.0064204 , -0.00310716,  0.00024792,
         0.00142647,  0.00284645, -0.00075836, -0.00086311,  0.0010776 ,
        -0.00036914, -0.00070547, -0.00281054,  0.0049522 ,  0.00405884,
        -0.00054775,  0.00157047]),
 array([0.00023344, 0.00018511, 0.00040193, 0.000278  , 0.00028319,
        0.00032295, 0.00033454, 0.00030908, 0.00028455, 0.00037811,
        0.0003192 , 0.00023363, 0.00049005, 0.00038302, 0.00038613,
        0.00034759, 0.00032509, 0.00011253, 0.00041535, 0.00030762,
        0.00026995, 0.00022049, 0.00034053, 0.00034293, 0.00028088,
        0.00032749, 0.00033736, 0.00040326, 0.00015504, 0.00017648,
        0.00033427, 0.00026488]))

In [34]:
# calc FST LPdom vs LBVwil
allel.average_weir_cockerham_fst(gt, subpops_LPdom_LBVwil, 50000, max_allele=None)

(0.04828047802549693,
 0.003444030899855149,
 array([0.0401801 , 0.05339936, 0.03538244, 0.03978789, 0.02734472,
        0.03137254, 0.03995392, 0.02544859, 0.03265747, 0.04124576,
        0.04138209, 0.04417723, 0.04128753, 0.04827041, 0.0413991 ,
        0.03367883, 0.04200036, 0.03711933, 0.04020126, 0.05144915,
        0.06217091, 0.13149092, 0.06820711, 0.05724607, 0.05644635,
        0.04244631, 0.04052648, 0.04503953, 0.0528119 , 0.06811598,
        0.05398709, 0.06226102]),
 array([0.04838264, 0.04793395, 0.04851768, 0.04836728, 0.0487496 ,
        0.04864605, 0.0483558 , 0.0488002 , 0.04859748, 0.04834402,
        0.04832292, 0.04824139, 0.04834097, 0.04810711, 0.04832824,
        0.048584  , 0.04830465, 0.0484576 , 0.0483659 , 0.04800554,
        0.0476618 , 0.04526214, 0.047443  , 0.04782042, 0.04783935,
        0.04829531, 0.04834878, 0.04820828, 0.04796021, 0.04742582,
        0.0479135 , 0.04764739]))

In [35]:
# calc FST LPfor vs LBVwil
allel.average_weir_cockerham_fst(gt, subpops_LPfor_LBVwil, 50000, max_allele=None)

(0.04851346470596654,
 0.002981110616211439,
 array([0.04414014, 0.05884193, 0.03534129, 0.03711501, 0.03121141,
        0.03771544, 0.04310359, 0.03644676, 0.03822501, 0.04629341,
        0.04115476, 0.04240368, 0.03895226, 0.05110186, 0.03874916,
        0.03839118, 0.0426089 , 0.0397378 , 0.04043964, 0.05201676,
        0.05565299, 0.12360883, 0.06344753, 0.05622497, 0.05321136,
        0.0453564 , 0.0394697 , 0.050939  , 0.05097256, 0.06451645,
        0.04615347, 0.05714954]),
 array([0.04856998, 0.04807158, 0.04884003, 0.04876864, 0.04895334,
        0.04876798, 0.04858289, 0.04878965, 0.04874398, 0.04849568,
        0.04865003, 0.04862015, 0.04874054, 0.04833669, 0.04873314,
        0.04875223, 0.04860663, 0.04869587, 0.04867832, 0.04830811,
        0.048193  , 0.04585483, 0.04792352, 0.04817358, 0.04826751,
        0.04852269, 0.04870297, 0.0483446 , 0.0483408 , 0.0478706 ,
        0.04849946, 0.04813958]))

In [18]:
meta = pd.read_csv("/home/daron/bioInf/wilding/popstructure/popdifferenciation/fst/ag1000g_wilding.meta.txt", sep='\t')
meta

,ox_code,population
0,AA0040-C,GHcol
1,AA0041-C,GHcol
2,AA0042-C,GHcol
3,AA0043-C,GHcol
4,AA0044-C,GHcol
...,...,...
1214,LBV127,LBVwil
1215,LBV130,LBVwil
1216,LBV132,LBVwil
1217,LBV133,LBVwil
